In [1]:
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import wandb

%load_ext autoreload
%autoreload all

from models import SiameseResNet
from datasets import DominanceDataset
from utils import read_images

In [11]:
images = read_images()

Loaded images from cache.


In [19]:
train_dataset = DominanceDataset(images, split="train")
test_dataset = DominanceDataset(images, split="test")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [20]:
run = wandb.init(
    project='corflow-dominance', 
    config={
        "architecture": "SiameseResNet18",
    }
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SiameseResNet() 
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_test_loss = np.inf
pbar = tqdm(range(100))
for epoch in pbar:
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []
    for imgs_l, imgs_r, labels in train_loader:
        imgs_l, imgs_r, labels = imgs_l.to(device), imgs_r.to(device), labels.to(device)

        outputs = model(imgs_l, imgs_r)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_train_loss = running_loss / len(train_loader)
    train_f1_score = f1_score(all_labels, all_preds, average='weighted')

    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs_l, imgs_r, labels in test_loader:
            imgs_l, imgs_r, labels = imgs_l.to(device), imgs_r.to(device), labels.to(device)
            
            outputs = model(imgs_l, imgs_r)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_test_loss = running_loss / len(test_loader)
    test_f1_score = f1_score(all_labels, all_preds, average='weighted')
    
    pbar.set_description(
        f'Loss: {avg_train_loss:.4f}/{avg_test_loss:.4f} | ' 
        f'F1 {train_f1_score:.4f}/{test_f1_score:.4f}'
    )

    wandb.log({
        "test_loss": avg_test_loss, "test_f1_score": test_f1_score,
        "train_loss": avg_train_loss, "train_f1_score": train_f1_score,
        "epoch": epoch
    })

    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        torch.save(model.state_dict(), 'weights/dominance.pth')

artifact = wandb.Artifact('model-artifact', type='model')
artifact.add_file('weights/dominance.pth')
wandb.log_artifact(artifact)

wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Loss: 0.2869/0.2136 | F1 0.9429/1.0000: 100%|██████████████████████████████████████████████████████████████████████████████████| 100/100 [01:17<00:00,  1.29it/s]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_f1_score,██████▇█▂▆▇▇▁██▇██▇████▂▇██▅▇▇█▅█▇▇▆▅▆▇█
test_loss,▁▁▁▁▁▁▂▂▂▁▁▂▄▁▁▁▁▁▂▁▁▁▁█▂▁▁▂▂▁▁▃▁▁▁▃▃▂▁▂
train_f1_score,▁▂▂▂▂▂▃▃▄▆▆▃▅▆█▇▆▃▇▅▆▇█▆▄▆█▇▇▃▇▇▅▆███▇█▇
train_loss,▃▂▂▂▂▃▂▂▂▁▂▂▂▂▁▁▄▂▂▄▁▁▁█▂▁▁▂▁▃▄▁▂▄▁▁▁▁▃▂
epoch,99
test_f1_score,1.0
test_loss,0.21355
train_f1_score,0.9429
train_loss,0.28693
